# **1. 라이브러리 임포트**
+  전국 태양광 발전소 시각화(좌표 통합 + 개별 클러스터, 지역별 고유 색상 자동 생성)하기 위함 

In [ ]:
# ====================================
# 🌞 전국 태양광 발전소 시각화 (Spiderfy 완전 비활성화 버전)
# ====================================

import os, json, time, html, webbrowser, re, colorsys
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import requests
from tqdm import tqdm
import folium
from folium.plugins import MarkerCluster
from folium import Element
import html
import json

# **2. 수집한 데이터 호출 및 전처리**
+ 데이터 전처리 방법
    + 엑셀 컬럼명에서 공백, 특수문자 제거
    + 전북’, ‘서울시’ 등 비표준 지역명을 ‘전라북도’, ‘서울특별시’로 통일(지역 이름 표준화)
    + 설비용량(숫자형 변환), 광역+세부 지역 결합 → 대표지역명 생성
    + 주소 컬럼으로 지오코딩 준비
        + 지오 코딩: 주소나 장소명 같은 고유명칭을 위도, 경도 좌표로 변환하는 기술

In [ ]:
# ===== 경로 / API =====
FILE_PATH   = r"C:\ESG_Project1\file\generator_file\HOME_발전설비_발전기별.xlsx"
CACHE_FILE  = r"C:\ESG_Project1\map\coord_cache.json"
OUTPUT_HTML = r"C:\ESG_Project1\map\solar_map.html"
KAKAO_API_KEY = "93c089f75a2730af2f15c01838e892d3"  # 본인(배경은) 키

# ===== 유틸: 컬럼/지역 표준화 =====
def clean_cols(cols: pd.Index) -> pd.Index:
    return (cols.str.replace('\ufeff', '', regex=False)
                .str.replace(r'\s+', ' ', regex=True)
                .str.strip())

PROVINCE_MAP = {
    "전북특별자치도": "전라북도", "전북": "전라북도",
    "전남": "전라남도", "경북": "경상북도", "경남": "경상남도",
    "충북": "충청북도", "충남": "충청남도",
    "서울시": "서울특별시", "부산시": "부산광역시", "대구시": "대구광역시",
    "인천시": "인천광역시", "광주시": "광주광역시", "대전시": "대전광역시",
    "울산시": "울산광역시", "세종시": "세종특별자치시",
    "제주도": "제주특별자치도", "강원특별자치도": "강원도",
}

def normalize_region(s: str) -> str:
    if pd.isna(s): return ""
    s = re.sub(r"\s+", "", str(s).strip())
    return PROVINCE_MAP.get(s, s)

def normalize_subregion(s: str) -> str:
    if pd.isna(s): return ""
    return re.sub(r"\s+", " ", str(s).strip())

# ===== 데이터 로드/정리 =====
df = pd.read_excel(FILE_PATH)
df.columns = clean_cols(df.columns)

region_col, subregion_col = '광역지역', '세부지역'
df['설비용량'] = pd.to_numeric(df.get('설비용량', 0), errors='coerce').fillna(0)
df['광역지역_norm'] = df[region_col].apply(normalize_region)
df['세부지역_norm']  = df[subregion_col].apply(normalize_subregion)
df['대표지역명'] = (df['광역지역_norm'] + " " + df['세부지역_norm']).str.strip()
df['주소'] = (df[subregion_col].astype(str)).str.strip()

# ===== 🔎 나쁜 라벨(빈값/문자 'nan' 등) 제거 규칙 =====
BAD_LABELS = {"", "nan", "None", "알수없음"}
def valid_region(x) -> bool:
    if x is None: return False
    s = str(x).strip()
    return s not in BAD_LABELS

# 정규화 후, 광역지역이 유효하지 않은 행 제거
df = df[df['광역지역_norm'].apply(valid_region)].copy()


c:\Users\baesa\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# **3. 지역별 색상 자동 생성**
+ 전처리한 데이터를 지도에 표시하기 위함  

In [3]:
#  ===== 🎨 지역별 자동 색상 (HSV 분할) =====
def _hsv_hex(h, s=0.85, v=0.9):
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return '#%02x%02x%02x' % (int(r*255), int(g*255), int(b*255))

unique_regions = sorted({str(r).strip() for r in df['광역지역_norm'] if valid_region(r)})
palette = [_hsv_hex(i / max(1, len(unique_regions))) for i in range(len(unique_regions))]
REGION_COLORS = dict(zip(unique_regions, palette))

def pick_region_color(region_norm: str) -> str:
    if not valid_region(region_norm):
        return "#7f7f7f"
    return REGION_COLORS.get(str(region_norm).strip(), "#7f7f7f")

# **4. 좌표 변환, 통합, 정보 매핑**
+ 카카오맵 API로 지오 코딩을 하여 전처리한 데이터 안의 주소를 변환(위도·경도로 변환)
+ 같은 위치의 발전소들을 하나의 점으로 합침
    + 각 좌표별로 발전소 수, 총 설비용량, 대표 지역명, 대표 광역 지역을 계산
+ 좌표를 통해 집계정보 매핑 (개별 발전소 팝업용으로 사용)
    + 개별 발전소 마커에서도 "해당 좌표의 요약 정보"를 바로 표시하기 위한 매핑 테이블로 사용 


In [4]:
# ===== 좌표 캐시 + 카카오 지오코딩 =====
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r", encoding="utf-8") as f:
        coords_cache = json.load(f)
else:
    coords_cache = {}

def get_coords_kakao(address: str):
    if address in coords_cache:
        return address, coords_cache[address]
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    try:
        r = requests.get(url, headers=headers, params=params, timeout=5)
        r.raise_for_status()
        data = r.json()
        if data.get('documents'):
            x = float(data['documents'][0]['x']); y = float(data['documents'][0]['y'])
            coords_cache[address] = [y, x]
        else:
            coords_cache[address] = [None, None]
    except Exception:
        coords_cache[address] = [None, None]
    return address, coords_cache[address]

targets = [a for a in df['주소'].dropna().unique() if a not in coords_cache]
if targets:
    print(f"📡 좌표 요청 대상: {len(targets)}건")
    with ThreadPoolExecutor(max_workers=8) as ex:
        futures = [ex.submit(get_coords_kakao, addr) for addr in targets]
        for _ in tqdm(as_completed(futures), total=len(futures), desc="좌표 변환"):
            _; time.sleep(0.05)
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(coords_cache, f, ensure_ascii=False, indent=2)

df['coords'] = df['주소'].map(coords_cache)
df[['위도','경도']] = pd.DataFrame(df['coords'].tolist(), index=df.index)
df = df.dropna(subset=['위도','경도'])

# ===== 좌표 통합 (대표광역이 유효하지 않은 행 제거 포함) =====
grouped = (
    df.groupby(['위도','경도'], as_index=False)
      .agg(
          발전소수=('발전기명','count'),
          총설비용량=('설비용량','sum'),
          대표지역명=('대표지역명', lambda x: x.value_counts().idxmax()),
          대표광역=('광역지역_norm', lambda x: x.value_counts().idxmax()),
      )
)
grouped = grouped[grouped['대표광역'].apply(valid_region)].copy()

📡 좌표 요청 대상: 332건


좌표 변환: 100%|██████████| 332/332 [00:17<00:00, 18.59it/s]


# **5. 지도 생성 및 조정** 
+ 한국 중심 좌표 기준으로 생성
    + 지도 타일 스타일: 밝은 회색(CartoDB positron)
+ 세부지역 단위(좌표 통합) 레이어
    + 각 지역을 대표 점으로 표시
    + 툴팁: 지역명 + 설비용량
    + 팝업: 발전소 수, 총 설비용량
+ 개별 발전소(클러스터) 레이어
    + 각 발전소를 동그라미 마커(CircleMarker)로 표시
    + 줌 레벨에 따라 자동 클러스터링
+ 지도 표시 범위 조정 & 레이어 컨트롤
    + 모든 마커가 화면에 들어오도록 자동 확대
    + 지도 우측 상단에서 레이어 on/off 가능하도록 컨트롤 조정함 
+ 지역 색상 범례
    + 왼쪽 아래에 고정하여 지역 이름과 색상이 한눈에 보이게 표시
        + 범례(legend): 지도, 차트, 그래프에서 기호, 색상, 무늬 등이 각각 무엇인지 설명하는 보조 안내 표지

In [5]:

# ====================================
# 4️⃣ 발전소 수 기반 반경 계산
# ====================================
MIN_RADIUS = 3
MAX_RADIUS = 12
min_count = grouped['발전소수'].min()
max_count = grouped['발전소수'].max()
def calc_radius(count, min_count, max_count):
    if max_count == min_count:
        return (MIN_RADIUS + MAX_RADIUS)/2
    ratio = (count - min_count) / (max_count - min_count)
    return MIN_RADIUS + ratio*(MAX_RADIUS - MIN_RADIUS)

# ====================================
# 5️⃣ 지도 생성
# ====================================
m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles="CartoDB positron")

# ① 세부지역 단위 마커 (발전소 수 기반 반경)
agg_layer = folium.FeatureGroup(name="세부지역 단위(지역별 색)", show=True).add_to(m)
bounds = []
for _, r in tqdm(grouped.iterrows(), total=len(grouped), desc="좌표 통합 마커"):
    lat, lon = float(r['위도']), float(r['경도'])
    name, cap, count = str(r['대표지역명']), float(r['총설비용량']), int(r['발전소수'])
    color = pick_region_color(r['대표광역'])
    popup_html = f"<b>{html.escape(name)}</b><br>발전소 수: {count}개<br>총 설비용량: {cap:.2f} MW"
    radius = calc_radius(count, min_count, max_count)
    folium.CircleMarker(
        location=[lat, lon], radius=radius,
        color=color, fill=True, fill_color=color, fill_opacity=0.85,
        popup=folium.Popup(popup_html, max_width=320),
        tooltip=f"{name} ({cap:.2f} MW)"
    ).add_to(agg_layer)
    bounds.append([lat, lon])

# ② 개별 발전소 클러스터
cluster_detail = MarkerCluster(name="개별 발전소", show=False).add_to(m)
for _, row in tqdm(df.iterrows(), total=len(df), desc="개별 발전소 마커"):
    lat, lon = float(row['위도']), float(row['경도'])
    color = pick_region_color(row.get('광역지역','기타'))
    pop = (
        f"<b>{html.escape(str(row.get('세부지역','')))}</b><br>"
        f"<b>회사명:</b> {html.escape(str(row.get('회사명','정보없음')))}<br>"
        f"<b>발전기명:</b> {html.escape(str(row.get('발전기명','정보없음')))}<br>"
        f"<b>발전원:</b> {html.escape(str(row.get('발전원','정보없음')))}<br>"
        f"<b>설비용량:</b> {float(row.get('설비용량',0)):.2f} MW"
    )
    folium.CircleMarker(
        location=[lat, lon], radius=5,
        color=color, fill=True, fill_color=color, fill_opacity=0.85,
        popup=folium.Popup(pop, max_width=320)
    ).add_to(cluster_detail)

if bounds:
    m.fit_bounds(bounds)

# ③ LayerControl
folium.LayerControl(collapsed=False).add_to(m)

# ====================================
# 6️⃣ Plotly 버블 추가 (세부지역 클릭 시)
# ====================================
grouped_json = grouped.to_dict(orient='records')
bubble_js = f"""
<div id="right-graph" style="position:fixed; top:10px; right:10px; width:480px; height:500px;
background:white; z-index:9999; padding:10px; border:2px solid black; overflow:auto;"></div>
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<script>
var df = {json.dumps(grouped_json, ensure_ascii=False)};

function drawGraph(region){{
    var filtered = df.filter(r=>r['대표광역']===region);
    if(filtered.length===0) {{
        document.getElementById('right-graph').innerHTML="<p>데이터 없음</p>";
        return;
    }}
    var x = filtered.map(r=>r['대표지역명']);
    var y = filtered.map(r=>r['발전소수']);
    var size = filtered.map(r=>r['총설비용량']);
    var minSize=10,maxSize=50;
    var minVal=Math.min(...size), maxVal=Math.max(...size);
    var scale = maxVal!==minVal ? (maxSize-minSize)/(maxVal-minVal) : 1;
    var bubbleSizes = size.map(s => minSize + (s - minVal)*scale);

    var trace={{x:x, y:y, text:x.map((v,i)=>v+"<br>총설비용량:"+size[i]+" MW"), 
                mode:"markers", marker:{{size:bubbleSizes, color:"steelblue", sizemode:"area", sizemin:5}}}};
    var layout={{title:region+" 세부지역 발전소 현황",
                xaxis:{{title:"세부지역",tickangle:-45}}, yaxis:{{title:"발전소 수"}},
                margin:{{l:40,r:10,t:40,b:100}}, hovermode:"closest"}};
    Plotly.newPlot("right-graph",[trace],layout,{{responsive:true}});
}}

// 클릭 이벤트
agg_layer.eachLayer(function(layer){{
    layer.on("click", function(e){{
        drawGraph(layer.options.tooltip.options['content'].split(' (')[0]);
    }});
}});
</script>
"""
m.get_root().html.add_child(Element(bubble_js))


개별 발전소 마커: 100%|██████████| 168879/168879 [01:27<00:00, 1930.92it/s]


# **6.생성된 지도 저장 및 HTML로 자동으로 열기**

In [6]:
# 저장 + 열기
m.save(OUTPUT_HTML)
print(f"✅ 지도 생성 완료: {OUTPUT_HTML}")
webbrowser.open(OUTPUT_HTML)

KeyboardInterrupt: 